In [1]:
import os
import pandas as pd
import numpy as np
from scipy.io import loadmat
from scipy.signal import welch
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings

In [2]:
# Ignore specific FutureWarning
warnings.filterwarnings("ignore", message="`rcond` parameter will change")

def extract_fetal_emg_features(emg_signal, fs, new_filename, window_len, overlap=0):
    # Rectify the EMG signal
    rectified_emg = np.abs(emg_signal)

    # Extract amplitude features
    features = {
        'time': new_filename,
        'amplitude_mean': np.mean(rectified_emg),
        'amplitude_median': np.median(rectified_emg),
        'amplitude_rms': np.sqrt(np.mean(rectified_emg**2)),
        'amplitude_std': np.std(rectified_emg),
        'amplitude_max': np.max(rectified_emg)
    }

    # Estimate the PSD using Welch's method
    freqs, psd = welch(rectified_emg, fs, nperseg=window_len, noverlap=overlap)

    # Extract frequency band powers
    features['low_power'] = np.trapz(psd[(freqs >= 25) & (freqs <= 60)], freqs[(freqs >= 25) & (freqs <= 60)])
    features['med_power'] = np.trapz(psd[(freqs >= 60) & (freqs <= 250)], freqs[(freqs >= 60) & (freqs <= 250)])
    features['high_power'] = np.trapz(psd[(freqs >= 250) & (freqs <= 500)], freqs[(freqs >= 250) & (freqs <= 500)])

    # Calculate relative powers
    total_power = np.trapz(psd, freqs)
    features['low_rel_power'] = features['low_power'] / total_power
    features['mid_rel_power'] = features['med_power'] / total_power
    features['high_rel_power'] = features['high_power'] / total_power

    # Extract dominant frequency
    frequencies = np.fft.fft(emg_signal)
    frequency_range = np.fft.fftfreq(len(emg_signal), 1 / fs)
    dominant_frequency_index = np.argmax(np.abs(frequencies[:len(emg_signal) // 2 + 1]))
    features['dominant_frequency'] = np.abs(frequency_range[dominant_frequency_index])

    # Extract timing features
    threshold = 0.1 * np.max(rectified_emg)
    onset_times, offset_times = find_emg_onset_offset(rectified_emg, threshold, fs)
    features['duration'] = np.median(np.diff(offset_times))

    return features

In [3]:
def find_emg_onset_offset(emg_signal, threshold, fs):
    # Find onset times
    above_threshold = emg_signal > threshold
    diff_above_threshold = np.diff(above_threshold.astype(int))
    onset_times = np.where(diff_above_threshold > 0)[0] + 1

    # Find offset times
    offset_times = np.where(diff_above_threshold < 0)[0] + 1

    # Remove offset times before the first onset time
    offset_times = offset_times[offset_times >= onset_times[0]]

    # If there are more onset times than offset times, remove the last onset time
    if len(onset_times) > len(offset_times):
        onset_times = onset_times[:-1]

    # Convert onset and offset times from samples to seconds
    onset_times = onset_times / fs
    offset_times = offset_times / fs

    return onset_times, offset_times

In [4]:
def capture_uco_windows_emg(emg_data, fs, start_time, uco_start_time, uco_end_time, additional_file_before=None, additional_start_time_before=None, additional_file_after=None, additional_start_time_after=None, window_len_sec=15):
    """
    Capture UCO time before and after windows and compute EMG features.

    Parameters
    ----------
    emg_data : array-like
        The EMG signal data.
    fs : int
        Sampling frequency of the EMG data.
    start_time : str
        Start time of the EMG data in format 'HH:MM:SS:%f %p'.
    uco_start_time : str
        UCO start time in format 'HH:MM:SS.%f %p'.
    uco_end_time : str
        UCO end time in format 'HH:MM:SS.%f %p'.
    additional_file_before : str, optional
        Path to the additional file needed for segments before UCO start time.
    additional_start_time_before : str, optional
        Start time of the additional file for segments before UCO start time.
    additional_file_after : str, optional
        Path to the additional file needed for segments after UCO end time.
    additional_start_time_after : str, optional
        Start time of the additional file for segments after UCO end time.
    window_len_sec : int
        Length of each window in seconds.

    Returns
    -------
    df : DataFrame
        DataFrame containing computed features for each window.
    """
    # Convert times to datetime objects
    start_datetime = datetime.strptime(start_time, '%I:%M:%S:%f %p')
    uco_start_datetime = datetime.strptime(uco_start_time, '%I:%M:%S.%f %p')
    uco_end_datetime = datetime.strptime(uco_end_time, '%I:%M:%S.%f %p')

    # Adjust for UCO times that cross midnight
    if uco_start_datetime < start_datetime:
        uco_start_datetime += timedelta(days=1)
    if uco_end_datetime < uco_start_datetime:
        uco_end_datetime += timedelta(days=1)
    
    # Calculate the difference in seconds between the start time and UCO times
    time_diff_start = (uco_start_datetime - start_datetime).total_seconds()
    time_diff_end = (uco_end_datetime - start_datetime).total_seconds()
    
    # Convert the time differences to sample indices
    uco_start_samples = int(time_diff_start * fs)
    uco_end_samples = int(time_diff_end * fs)
    
    # Define window length in samples
    window_len = window_len_sec * fs
    
    # Calculate the start and end sample indices for the desired windows
    start_sample_before = max(0, uco_start_samples - 75 * 60 * fs)
    end_sample_before = uco_start_samples - 60 * 60 * fs
    start_sample_uco = uco_start_samples
    end_sample_uco = uco_end_samples
    start_sample_after = uco_end_samples
    end_sample_after = min(len(emg_data), uco_end_samples + 60 * window_len)

    print(f"time diff start: {time_diff_start}")
    print(f"time diff end: {time_diff_end}")
    print(f"start sample before UCO: {start_sample_before}")
    print(f"end sample before UCO: {end_sample_before}")
    print(f"start sample after UCO: {start_sample_after}")
    print(f"end sample after UCO: {end_sample_after}")

    data = {
        'time': [],
        'amplitude_mean': [],
        'amplitude_median': [],
        'amplitude_rms': [],
        'amplitude_std': [],
        'amplitude_max': [],
        'low_power': [],
        'med_power': [],
        'high_power': [],
        'low_rel_power': [],
        'mid_rel_power': [],
        'high_rel_power': [],
        'dominant_frequency': [],
        'duration': [],
        'label': []
    }

    def process_windows(signal_data, start_sample, end_sample, label):
        for start in range(start_sample, end_sample, window_len):
            end = start + window_len
            if end > len(signal_data):
                break  # Ensure the last window doesn't exceed the data length

            segment = signal_data[start:end].flatten()

            try:
                features = extract_fetal_emg_features(segment, fs, start / fs, window_len, 0)

                data['time'].append(start / fs)
                data['amplitude_mean'].append(features['amplitude_mean'])
                data['amplitude_median'].append(features['amplitude_median'])
                data['amplitude_rms'].append(features['amplitude_rms'])
                data['amplitude_std'].append(features['amplitude_std'])
                data['amplitude_max'].append(features['amplitude_max'])
                data['low_power'].append(features['low_power'])
                data['med_power'].append(features['med_power'])
                data['high_power'].append(features['high_power'])
                data['low_rel_power'].append(features['low_rel_power'])
                data['mid_rel_power'].append(features['mid_rel_power'])
                data['high_rel_power'].append(features['high_rel_power'])
                data['dominant_frequency'].append(features['dominant_frequency'])
                data['duration'].append(features['duration'])
                data['label'].append(label)

                plt.close()
            except Exception as e:
                print(f"Error processing segment: {e}")

    # Process windows 1 hour and 15 minutes to 1 hour before UCO start time and label as -1
    if start_sample_before < end_sample_before:
        process_windows(emg_data, start_sample_before, end_sample_before, label=-1)
    elif additional_file_before and additional_start_time_before:
        additional_data = loadmat(additional_file_before)['save_data'].flatten()
        new_start_datetime_before = datetime.strptime(additional_start_time_before, '%I:%M:%S:%f %p')
        
        # Adjust for additional start time crossing midnight
        if uco_start_datetime < new_start_datetime_before:
            uco_start_datetime += timedelta(days=1)
        
        additional_time_diff_start = (uco_start_datetime - new_start_datetime_before).total_seconds() - 75 * 60
        additional_start_samples = int(additional_time_diff_start * fs)
        process_windows(additional_data, additional_start_samples, additional_start_samples + 60 * window_len, label=-1)
    
    # Process windows during UCO and label as 0
    process_windows(emg_data, start_sample_uco, end_sample_uco, label=0)
    
    # Process windows after UCO end time and label as 1
    remaining_duration = len(emg_data) - start_sample_after
    if remaining_duration >= 15 * 60 * fs:
        process_windows(emg_data, start_sample_after, end_sample_after, label=1)
    else:
        process_windows(emg_data, start_sample_after, len(emg_data), label=1)
        if additional_file_after and additional_start_time_after:
            additional_data = loadmat(additional_file_after)['save_data'].flatten()
            new_start_datetime_after = datetime.strptime(additional_start_time_after, '%I:%M:%S:%f %p')
            
            # Adjust for additional start time crossing midnight
            if new_start_datetime_after < start_datetime:
                new_start_datetime_after += timedelta(days=1)
            
            additional_start_samples = 0  # Start from the beginning of the additional file
            additional_end_samples = int((15 * 60 * fs - remaining_duration))
            process_windows(additional_data, additional_start_samples, additional_end_samples, label=1)

    df = pd.DataFrame(data)
    return df


In [5]:
# Define the parameters
main_file = r'D:\IEEE Sensor\IEEE Sensor Matlab Data\EMG\21044\Clean Data EMGCUsersvaishDownloadsFetal EEG-20231204T232927Z-001Fetal EEGFYP 2023CODE BACK UPRaw Data2104421044_1-06-2021 9_25_16.2 AM_ UCO file Channel 2.mat'
handled_folder_path = r'D:\IEEE Sensor\IEEE Sensor Xlsx Data\EMG_handled'
fs = 2000  # Sampling frequency
start_time = '09:25:16:2 AM'  # Start time of the data
uco_start_time = '09:54:34.055758 AM'  # UCO start time
uco_end_time = '10:09:31.555758 AM'  # UCO end time

additional_file_before = r'D:\IEEE Sensor\IEEE Sensor Matlab Data\EMG\21044\Clean Data EMGCUsersvaishDownloadsFetal EEG-20231204T232927Z-001Fetal EEGFYP 2023CODE BACK UPRaw Data2104421044_1-06-2021 9_25_16.2 AM_ UCO file Channel 1.mat'  # Path to additional file for before UCO
additional_start_time_before = '11:52:09:0 PM'  # Start time of the additional file for before UCO
additional_file_after = r'D:\IEEE Sensor\IEEE Sensor Matlab Data\EMG\21126\Clean Data EMGCUsersvaishDownloadsFetal EEG-20231204T232927Z-001Fetal EEGFYP 2023CODE BACK UPRaw Data2112621126_20-08-2021 9_05_07.3 AM Channel 1.mat'  # Path to additional file for after UCO
additional_start_time_after = '09:05:07:3 AM'  # Start time of the additional file for after UCO

# Make sure the processed folder exists
if not os.path.exists(handled_folder_path):
    os.makedirs(handled_folder_path)

# Load the main file
main_data = loadmat(main_file)
emg_data = main_data['save_data'].flatten()

# Capture UCO windows and compute features
df = capture_uco_windows_emg(emg_data, fs, start_time, uco_start_time, uco_end_time, additional_file_before, additional_start_time_before, additional_file_after, additional_start_time_after)

# Define new save path
folder_name = os.path.basename(os.path.dirname(main_file))
new_save_folder = os.path.join(handled_folder_path, folder_name + '_handled')
if not os.path.exists(new_save_folder):
    os.makedirs(new_save_folder)

excel_file_name = os.path.basename(main_file)[:-4] + '.xlsx'  # Change the extension to .xlsx
new_excel_file_path = os.path.join(new_save_folder, excel_file_name)

df.to_excel(new_excel_file_path, index=False)

print("Processing completed! All processed files have been saved.")

time diff start: 1757.855758
time diff end: 2655.355758
start sample before UCO: 0
end sample before UCO: -3684289
start sample after UCO: 5310711
end sample after UCO: 7110711
Processing completed! All processed files have been saved.
